# Nzpy simple load and unload example

### Setup

pip install nzpy

### Make connection

In [1]:
import nzpy
import pandas as pd

conn = nzpy.connect(user="admin", password="password",host='localhost', port=5480, database="DB1", securityLevel=0)

### Drop table if exists

In [2]:
cursor = conn.cursor()
try:
    cursor.execute("drop table customerAddress")
except nzpy.DatabaseError as e:
    pass
except Exception as e:
    pass


### Create table and insert rows

In [3]:
cursor.execute("create table customerAddress(Id int, Name varchar(10), Address varchar(50), Email varchar(20) )")
cursor.execute("insert into customerAddress values (1,'Jack','High street London', 'jack4321@ibm.com')")
cursor.execute("insert into customerAddress values (2,'Tom', 'Park street NY','tom1234@ibm.com')")
cursor.execute("insert into customerAddress values (3,'James', 'MG street SG','james678@ibm.com')")

### Select rows from table using fetchall()

In [4]:
cursor.execute("select * from customerAddress")
results = cursor.fetchall()

for c1,c2,c3,c4 in results:
    print("colums1 = %s" % (c1))
    print("colums2 = %s" % (c2))
    print("colums3 = %s" % (c3))


colums1 = 1
colums2 = Jack
colums3 = High street London
colums1 = 2
colums2 = Tom
colums3 = Park street NY
colums1 = 3
colums2 = James
colums3 = MG street SG


### Select rows from table using pandas read_sql_query()

In [5]:
result = pd.read_sql_query("select * from customerAddress", conn)
result.columns = [c.decode().lower() for c in result.columns]
result


,id,name,address,email
0,1,Jack,High street London,jack4321@ibm.com
1,2,Tom,Park street NY,tom1234@ibm.com
2,3,James,MG street SG,james678@ibm.com


### Create external table and dump data to csv file

In [6]:
with conn.cursor() as cursor:
    cursor.execute("create external table 'cust.csv' using ( remotesource 'python' delim ',' logDir '.') as select * from customerAddress")

df = pd.read_csv('cust.csv', names=['id', 'name', 'address', 'email'])
df


,id,name,address,email
0,1,Jack,High street London,jack4321@ibm.com
1,2,Tom,Park street NY,tom1234@ibm.com
2,3,James,MG street SG,james678@ibm.com


### Read data from csv file and load into table

In [7]:
with conn.cursor() as cursor:
    cursor.execute("insert into customerAddress select * from external 'cust.csv' using (delim ',' remotesource 'python' logDir '.')")

In [8]:
result = pd.read_sql_query("select * from customerAddress", conn)
result.columns = [c.decode().lower() for c in result.columns]
result

,id,name,address,email
0,1,Jack,High street London,jack4321@ibm.com
1,2,Tom,Park street NY,tom1234@ibm.com
2,3,James,MG street SG,james678@ibm.com
3,1,Jack,High street London,jack4321@ibm.com
4,2,Tom,Park street NY,tom1234@ibm.com
5,3,James,MG street SG,james678@ibm.com
